In [2]:
import pandas as pd

In [3]:
ICD10_MAP = {
    'C50': 'Breast',
    'C51': 'Vulva',
    'C52': 'Vagina',
    'C53': 'Cervix Uteri',
    'C54': 'Corpus Uteri',
    'C55': 'Uterus',
    'C56': 'Ovary',
    'C57': 'Other and Unspecified Female Genital Organs',
}

In [15]:
df = pd.read_csv('heisin_dag.csv')

In [16]:
LEVEL = 'level'
DIAG = 'diag_icd10'
FEMAL_CANCER_TYPE = 'cancer_type'
df[FEMAL_CANCER_TYPE] =df.apply(lambda x: ICD10_MAP.get(str(x[DIAG])[:3], None), axis=1)

primary_fc = df[(df[LEVEL] == 'Primary/main diagnosis') & (df[DIAG].str.contains('C5', na=False))]

In [17]:
dates = pd.read_csv('bc_hesin.csv', parse_dates=['epistart'])[['dnx_hesin_id', 'epistart']] 
primary_fc = primary_fc.merge(dates, on='dnx_hesin_id', how='left')
primary_fc.info(show_counts=True)

/tmp/ipykernel_2592/3330282795.py:1: DtypeWarning: Columns (3,19,36) have mixed types. Specify dtype option on import or set low_memory=False.
  dates = pd.read_csv('bc_hesin.csv', parse_dates=['epistart'])[['dnx_hesin_id', 'epistart']]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142992 entries, 0 to 142991
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   dnx_hesin_diag_id  142992 non-null  object        
 1   dnx_hesin_id       142992 non-null  object        
 2   eid                142992 non-null  int64         
 3   ins_index          142992 non-null  int64         
 4   arr_index          142992 non-null  int64         
 5   level              142992 non-null  object        
 6   diag_icd9          0 non-null       object        
 7   diag_icd9_nb       0 non-null       float64       
 8   diag_icd10         142992 non-null  object        
 9   diag_icd10_nb      32 non-null      object        
 10  cancer_type        142992 non-null  object        
 11  epistart           142831 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 13.1+ MB


In [27]:
pivot_df = primary_fc.pivot_table(index='eid', 
                          columns=FEMAL_CANCER_TYPE, 
                          values='epistart', 
                          aggfunc='min', 
                          fill_value=None).reset_index()
pivot_df['number_cancer'] = pivot_df[ICD10_MAP.values()].count(axis=1)
# Display result
pivot_df.groupby('number_cancer').nunique('eid')

cancer_type,eid,Breast,Cervix Uteri,Corpus Uteri,Other and Unspecified Female Genital Organs,Ovary,Uterus,Vagina,Vulva
number_cancer,,,,,,,,,
1,19401,6280,2,29,20,1290,2,0,3
2,638,414,27,302,146,314,18,6,26
3,49,38,8,37,17,25,18,2,2
4,5,5,2,5,2,2,4,0,0


In [32]:
sum_of_true_values = pivot_df.drop(columns='eid').count()
print("Sum of True values per cancer type:")
print(sum_of_true_values)

Sum of True values per cancer type:
cancer_type
Breast                                         18348
Cervix Uteri                                      39
Corpus Uteri                                     377
Other and Unspecified Female Genital Organs      188
Ovary                                           1811
Uterus                                            42
Vagina                                             8
Vulva                                             31
number_cancer                                  20093
dtype: int64


In [33]:
pivot_df.to_csv('cleaned_diagnosis_data.csv')